# 2.00 Object Relational Mapping
This notebook demonstrates how we can use object relational mapping with Postgres.

It uses sqlalchemy as ORM. Install via `pip3 install sqlalchemy` or better, to install alongside the flask plugin use
`pip3 install flask_sqlalchemy`

In [1]:
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

In [2]:
# use optional echo=True to output commands
db = sqlalchemy.create_engine('postgresql://localhost/cs6', echo=True)

In [3]:
Base = declarative_base()

In [4]:
class Animal(Base):
    __tablename__ = 'animals'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    type = Column(String)
    
    def __repr__(self):
        return '{}({})'.format(self.name, self.type)

In [5]:
Animal(name='Tux', type='penguin')

Tux(penguin)

In [6]:
Base.metadata.create_all(db)

2019-11-18 23:32:45,696 INFO sqlalchemy.engine.base.Engine select version()
2019-11-18 23:32:45,697 INFO sqlalchemy.engine.base.Engine {}
2019-11-18 23:32:45,699 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-11-18 23:32:45,699 INFO sqlalchemy.engine.base.Engine {}
2019-11-18 23:32:45,701 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-11-18 23:32:45,702 INFO sqlalchemy.engine.base.Engine {}
2019-11-18 23:32:45,703 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-11-18 23:32:45,703 INFO sqlalchemy.engine.base.Engine {}
2019-11-18 23:32:45,704 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-11-18 23:32:45,704 INFO sqlalchemy.engine.base.Engine {}
2019-11-18 23:32:45,706 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

## 2.01 Communicating with the database
To communicate with the database, we need to create a session object

In [7]:
from sqlalchemy.orm import sessionmaker

In [8]:
Session = sessionmaker(bind=db)

In [9]:
session = Session()

Let's say we want to add two new penguins to our database. We can use the above defined class to create instances.

In [10]:
tux = Animal(name='Tux', type='penguin')
tango = Animal(name='Tango', type='penguin')

In the next step, we add these objects to the session. So far no SQL queries have been issued to the database.

In [11]:
session.add(tux)
session.add(tango)

However, when calling `commit` all changes in the session are flushed to the database.

In [12]:
session.commit()

2019-11-18 23:32:48,031 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-18 23:32:48,034 INFO sqlalchemy.engine.base.Engine INSERT INTO animals (name, type) VALUES (%(name)s, %(type)s) RETURNING animals.id
2019-11-18 23:32:48,037 INFO sqlalchemy.engine.base.Engine {'name': 'Tux', 'type': 'penguin'}
2019-11-18 23:32:48,041 INFO sqlalchemy.engine.base.Engine INSERT INTO animals (name, type) VALUES (%(name)s, %(type)s) RETURNING animals.id
2019-11-18 23:32:48,043 INFO sqlalchemy.engine.base.Engine {'name': 'Tango', 'type': 'penguin'}
2019-11-18 23:32:48,045 INFO sqlalchemy.engine.base.Engine COMMIT


To retrieve data from the database, we can use sqlalchemy as well:

In [13]:
for animal in session.query(Animal).order_by(Animal.name):
    print(animal.name, animal.type)

2019-11-18 23:32:48,647 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-18 23:32:48,649 INFO sqlalchemy.engine.base.Engine SELECT animals.id AS animals_id, animals.name AS animals_name, animals.type AS animals_type 
FROM animals ORDER BY animals.name
2019-11-18 23:32:48,649 INFO sqlalchemy.engine.base.Engine {}
Tango penguin
Tango penguin
Tango penguin
Tango penguin
Tux penguin
Tux penguin
Tux penguin
Tux penguin


For a complete tutorial on how to use SQLAlchemy accessors see https://docs.sqlalchemy.org/en/13/orm/tutorial.html

## 2.02 Relationships

Imagine we want to write a simple to do list for users. How could we do that using a database?


==> This is a 1 User <=> N todos relationship

More on relationships: <https://docs.sqlalchemy.org/en/13/orm/relationships.html>

In [14]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [15]:
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    todos = relationship('Todo', back_populates='user')

In [16]:
class Todo(Base):
    __tablename__ = 'todos'
    
    id = Column(Integer, primary_key=True)
    note = Column(String)
    
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship('User', back_populates='todos')

Create necessary tables

In [17]:
Base.metadata.create_all(db)

2019-11-18 23:32:51,257 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2019-11-18 23:32:51,258 INFO sqlalchemy.engine.base.Engine {'name': 'animals'}
2019-11-18 23:32:51,261 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2019-11-18 23:32:51,261 INFO sqlalchemy.engine.base.Engine {'name': 'users'}
2019-11-18 23:32:51,263 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2019-11-18 23:32:51,264 INFO sqlalchemy.engine.base.Engine {'name': 'todos'}


Now we can start retrieving a user

In [18]:
tux = User(name='Tux')

In [19]:
tux.todos.append(Todo(note='Go fishing'))
tux.todos.append(Todo(note='Go see sealion'))

In [20]:
session.add(tux)

In [21]:
session.commit()

2019-11-18 23:32:53,474 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name) VALUES (%(name)s) RETURNING users.id
2019-11-18 23:32:53,475 INFO sqlalchemy.engine.base.Engine {'name': 'Tux'}
2019-11-18 23:32:53,478 INFO sqlalchemy.engine.base.Engine INSERT INTO todos (note, user_id) VALUES (%(note)s, %(user_id)s) RETURNING todos.id
2019-11-18 23:32:53,478 INFO sqlalchemy.engine.base.Engine {'note': 'Go fishing', 'user_id': 1}
2019-11-18 23:32:53,481 INFO sqlalchemy.engine.base.Engine INSERT INTO todos (note, user_id) VALUES (%(note)s, %(user_id)s) RETURNING todos.id
2019-11-18 23:32:53,481 INFO sqlalchemy.engine.base.Engine {'note': 'Go see sealion', 'user_id': 1}
2019-11-18 23:32:53,483 INFO sqlalchemy.engine.base.Engine COMMIT


Again, we can make use of SQLAlchemy's builtin mechanisms to retrieve data as objects!

In [23]:
for todo in session.query(Todo):
    print('{} has "{}" to do!'.format(todo.user.name, todo.note))

2019-11-18 23:34:50,544 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-18 23:34:50,545 INFO sqlalchemy.engine.base.Engine SELECT todos.id AS todos_id, todos.note AS todos_note, todos.user_id AS todos_user_id 
FROM todos
2019-11-18 23:34:50,546 INFO sqlalchemy.engine.base.Engine {}
2019-11-18 23:34:50,549 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name 
FROM users 
WHERE users.id = %(param_1)s
2019-11-18 23:34:50,550 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
Tux has "Go fishing" to do!
Tux has "Go see sealion" to do!
